# Creating a zarr file for L3 Rrs

In this tutorial, we will create a zarr file. This is a cloud-optimized (chunked) data cube and can be easily be loaded with `xarray.open_zarr()`.

## Import modules

In [1]:
import cartopy.crs as ccrs
import earthaccess
import h5netcdf
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd

Set (and persist to your user profile on the host, if needed) your Earthdata Login credentials.

In [2]:
auth = earthaccess.login(persist=True)

## L3 File Structure

At Level-3 there are binned (B) and mapped (M) products available for OCI. The L3M remote sensing reflectance (Rrs) files contain global maps of Rrs. 

In [3]:
tspan = ("2024-05-01", "2024-05-08")
results = earthaccess.search_data(
    short_name="PACE_OCI_L3M_RRS_NRT",
    temporal=tspan,
#    count=1,
)

In [6]:
# We can look at the urls
[result.data_links() for result in results]

[['https://obdaac-tea.earthdatacloud.nasa.gov/ob-cumulus-prod-public/PACE_OCI.20240430_20240507.L3m.8D.RRS.V2_0.Rrs.1deg.NRT.nc'],
 ['https://obdaac-tea.earthdatacloud.nasa.gov/ob-cumulus-prod-public/PACE_OCI.20240430_20240507.L3m.8D.RRS.V2_0.Rrs.4km.NRT.nc'],
 ['https://obdaac-tea.earthdatacloud.nasa.gov/ob-cumulus-prod-public/PACE_OCI.20240430_20240507.L3m.8D.RRS.V2_0.Rrs.0p1deg.NRT.nc'],
 ['https://obdaac-tea.earthdatacloud.nasa.gov/ob-cumulus-prod-public/PACE_OCI.20240501.L3m.DAY.RRS.V2_0.Rrs.1deg.NRT.nc'],
 ['https://obdaac-tea.earthdatacloud.nasa.gov/ob-cumulus-prod-public/PACE_OCI.20240501.L3m.DAY.RRS.V2_0.Rrs.0p1deg.NRT.nc'],
 ['https://obdaac-tea.earthdatacloud.nasa.gov/ob-cumulus-prod-public/PACE_OCI.20240501.L3m.DAY.RRS.V2_0.Rrs.4km.NRT.nc'],
 ['https://obdaac-tea.earthdatacloud.nasa.gov/ob-cumulus-prod-public/PACE_OCI.20240501_20240531.L3m.MO.RRS.V2_0.Rrs.1deg.NRT.nc'],
 ['https://obdaac-tea.earthdatacloud.nasa.gov/ob-cumulus-prod-public/PACE_OCI.20240501_20240531.L3m.MO.RR

In [4]:
paths = earthaccess.open(results)

QUEUEING TASKS | :   0%|          | 0/33 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/33 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/33 [00:00<?, ?it/s]

OCI L3 data do not have any groups, so we can open the dataset without the `group` argument.
Let's take a look at one of these files. Not just any one; we will search for the one that is a
high resolution map.

In [5]:
for item in paths:
    dataset = xr.open_dataset(item)
    if dataset.sizes["lat"] == 4320:
        break
dataset

<xarray.Dataset> Size: 27GB
Dimensions:     (wavelength: 184, lat: 4320, lon: 8640, rgb: 3,
                 eightbitcolor: 256)
Coordinates:
  * wavelength  (wavelength) float64 1kB 339.0 341.0 344.0 ... 714.0 717.0 719.0
  * lat         (lat) float32 17kB 89.98 89.94 89.9 ... -89.9 -89.94 -89.98
  * lon         (lon) float32 35kB -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
Dimensions without coordinates: rgb, eightbitcolor
Data variables:
    Rrs         (lat, lon, wavelength) float32 27GB ...
    palette     (rgb, eightbitcolor) uint8 768B ...
Attributes: (12/64)
    product_name:                      PACE_OCI.20240430_20240507.L3m.8D.RRS....
    instrument:                        OCI
    title:                             OCI Level-3 Standard Mapped Image
    project:                           Ocean Biology Processing Group (NASA/G...
    platform:                          PACE
    source:                            satellite observations from OCI-PACE
    ...                                ...
    identifier_product_doi:            10.5067/PACE/OCI/L3M/RRS/2.0
    keywords:                          Earth Science > Oceans > Ocean Optics ...
    keywords_vocabulary:               NASA Global Change Master Directory (G...
    data_bins:                         12691623
    data_minimum:                      -0.010000004
    data_maximum:                      0.10000004

Notice that OCI L3M data has `lat`, `lon`, and `wavelength` coordinates, so it's easy to slice
out a bounding box and map the "Rrs" variable at a given wavelength.

In [ ]:
rrs_slice = dataset["Rrs"].sel({"lat": slice(-25, -45), "lon": slice(10, 30)})
rrs_slice_442 = rrs_slice.sel({"wavelength": 442}, method="nearest")
rrs_slice_442

In [ ]:
fig = plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())
im = rrs_slice_442.plot(cmap="viridis", robust=True, ax=ax)
ax.gridlines(draw_labels={"left": "y", "bottom": "x"})
ax.coastlines()
plt.show()

Also becuase the L3M variables have `lat` and `lon` coordinates, it's possible to stack multiple granules along a new dimension that corresponds to time.
Instead of `xr.open_dataset`, we use `xr.open_mfdataset` to create a single `xarray.Dataset` (the "mf" in `open_mfdataset` stands for multiple files) from an array of paths.

Rather than searching through results for particular resolutions though, we need to augment the CMR query using information
build into the granule name. Take a look at the attribute on the previous dataset.

In [ ]:
dataset.attrs["product_name"]

We will use a new search filter available in `search_data`: the `granule_name` argument accepts strings with the "*" wildcard. We need this to distinguish daily ("DAY") from eight-day ("8D") composites, as well as to get the desired 0.1 degree resolution projections.

In [ ]:
results = earthaccess.search_data(
    short_name="PACE_OCI_L3M_CHL_NRT",
    temporal=tspan,
    granule_name="*.DAY.*.0p1deg.*",
)

In [ ]:
paths = earthaccess.open(results)

The `paths` list is sorted temporally by default, which means the shape of the `paths` array specifies the way we need to tile the files together into larger arrays. We specify `combine="nested"` to combine the files according to the shape of the array of files (or file-like objects), even though `paths` is not a "nested" list in this case. The `concat_dim="date"` argument generates a new dimension in the combined dataset, because "date" is not an existing dimension in the individual files.

In [ ]:
dataset = xr.open_mfdataset(
    paths,
    combine="nested",
    concat_dim="date",
)

Add a date dimension using the dates from the netCDF files.

In [ ]:
dates = [xr.open_dataset(a).attrs["time_coverage_end"] for a in paths]
dt = pd.to_datetime(dates)
dataset = dataset.assign_coords(date=dt.values)
dataset

A common reason to generate a single dataset from multiple, daily images is to create a composite. Compare the map from a single day ...

In [ ]:
chla = np.log10(dataset["chlor_a"])
chla.attrs.update(
    {
        "units": f'log({dataset["chlor_a"].attrs["units"]})',
    }
)
im = chla.sel(date = "2024-05-02").plot(aspect=2, size=4, cmap="GnBu_r")

... to a map of average values, skipping "NaN" values that result from clouds and the OCI's tilt maneuver.

In [ ]:
chla_avg = chla.mean("date", keep_attrs=True)
im = chla_avg.plot(aspect=2, size=4, cmap="GnBu_r")

We can also create a time series of mean values over the whole region.

In [ ]:
chla_avg = chla.mean(dim=["lon", "lat"], keep_attrs=True)
im = chla_avg.plot(linestyle='-', marker='o', color='b')

[back to top](#Contents)

<div class="alert alert-info" role="alert">

You have completed the notebook on OCI file structure.

</div>